In [1]:
# @markdown # **Step 0.** _Removing sample_data_

!rm -rf /content/sample_data

In [2]:
# @markdown # **Step 1.** _Installing dependencies_

from google.colab import output

%cd /content

# Installing dependencies

!pip install g4f[all] pyngrok

# Clearing the output

output.clear()

In [ ]:
# @markdown # **Step 2.** _Launch of G4F_

import os
import sys
import signal
import subprocess

from google.colab import userdata
from pyngrok import ngrok, conf

%cd /content


def signal_handler(signal, frame) -> None:
    print("\n")
    print("Ctrl+C pressed. Aborting.")
    print("Come back soon!")

    sys.exit(1)


def open_tunnel() -> None:
    print("Opening of the tunnel...")

    ngrok.kill()

    ngrok_token = userdata.get("NGROK_TOKEN")
    ngrok_domain = userdata.get("NGROK_DOMAIN")

    ngrok_config = conf.PyngrokConfig(
        auth_token = ngrok_token
    )

    server = ngrok.connect(
        9090,
        pyngrok_config = ngrok_config,
        bind_tls = True,
        domain = ngrok_domain
    ).public_url

    print(f"The tunnel is open at the link: {server}")


def bind_signal() -> None:
    signal.signal(signal.SIGINT, signal_handler)
    print("Press Ctrl+C to exit.")


def start_ngrok() -> None:
    open_tunnel()
    bind_signal()

    command = f"python -m g4f.cli gui -port 9090 -debug"
    environment = os.environ.copy()

    subprocess.run(
        command,
        shell = True,
        env = environment
    )

    signal.pause()


start_ngrok()